In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
df_all = pd.read_csv("/kaggle/input/book-recommendation-dataset/Books.csv")
book_rating_all = pd.read_csv("/kaggle/input/book-recommendation-dataset/Ratings.csv")
book_users_all = pd.read_csv("/kaggle/input/book-recommendation-dataset/Users.csv")
df = df_all.iloc[:100000, :]
book_rating = book_rating_all.iloc[:100000, :]
book_users = book_users_all.iloc[:100000, :]
book_ratings = book_rating.iloc[:, 0:3]
book_ratings.rename(columns={"User-ID" : "user_id", "Book-Rating" : "book_rating"}, inplace=True)
df = pd.merge(df, book_ratings, on="ISBN")
df = df.reindex(columns=["ISBN", "user_id", "Book-Title", "Book-Author", "Year-Of-Publication", "Publisher", "book_rating", "Image-URL-S", "Image-URL-M", "Image-URL-L"])
print (df.shape)

In [ ]:
# visualisation
df.groupby('Book-Title')['book_rating'].describe().sort_values("mean", ascending = False).head(10)

In [ ]:
df["count"] = df.groupby('Book-Title')['book_rating'].transform("count")
df["more_than_one"] = np.where(df["count"] > 1, 1, 0)
df = df[df["more_than_one"] == 1]
df = df.drop("count", 1)
df = df.drop("more_than_one", axis = 1)
print (df.shape)

In [ ]:
ratings_df_mean = df.groupby('Book-Title')['book_rating'].describe()['mean']
ratings_df_count = df.groupby('Book-Title')['book_rating'].describe()['count']
ratings_mean_count_df = pd.concat([ratings_df_count, ratings_df_mean], axis = 1)
ratings_mean_count_df['mean'].plot(bins=100, kind='hist', color = 'r') 

In [ ]:
ratings_mean_count_df['count'].plot(bins=100, kind='hist', color = 'r') 

In [ ]:
userid_booktitle_matrix = df.pivot_table(index = 'user_id', columns = 'Book-Title', values = 'book_rating')


In [ ]:
userid_booktitle_matrix.sample()

In [ ]:
selected = "stardust"

selection = userid_booktitle_matrix[selected]

# Let's calculate the correlations
selection_correlations = pd.DataFrame(userid_booktitle_matrix.corrwith(selection), columns=['Correlation'])
selection_correlations = selection_correlations.join(ratings_mean_count_df['count'])
                                      
selection_correlations.dropna(inplace=True)
# Let's sort the correlations vector
                                      
selection_correlations[selection_correlations['count']>0].sort_values('Correlation',ascending=False).head()

In [ ]:


bookcorrelatons = userid_booktitle_matrix.corr(method = "pearson" , min_periods = 2)


In [ ]:
data = {'Book-Title': ["A Bend in the Road", "stardust" ], 'book_rating': [5, 1]}
myRatings = pd.DataFrame.from_dict(data)

similar_book_list = pd.Series()
for i in range(0, 2):
    similar_book = bookcorrelatons[myRatings['Book-Title'][i]].dropna() # Get same books with same ratings
    similar_book = similar_book.map(lambda x: x * myRatings['book_rating'][i]) # Scale the similarity by your given ratings
    similar_books_list = similar_book_list.append(similar_book)
similar_books_list.sort_values(inplace = True, ascending = False)
print (similar_books_list.head(10))